In [45]:
!pip install river

In [46]:
!pip install -U numpy

In [73]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib
import numpy as np

In [74]:
data_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/7205/all_train_data.txt")

In [75]:
data_df

,xpos,ypos,coin,emnemy,shot,enemy/shot,level,keyX,keyY,respawn_enemy,"""respawn_coin"""
0,415.25,344.25,1,0,1,1,1,3,4,5,6
1,317.60,178.00,7,6,6,0,1,13,13,6,7
2,344.20,218.60,6,6,8,2,2,21,16,11,11
3,297.30,387.30,2,8,10,2,3,13,10,13,7
4,354.70,372.60,4,8,9,1,3,17,10,8,4
...,...,...,...,...,...,...,...,...,...,...,...
382,472.50,536.00,0,9,11,2,4,8,0,9,0
383,385.00,536.00,2,7,8,1,5,15,0,12,7
384,227.50,536.00,0,7,8,1,5,14,0,7,0
385,435.40,536.00,1,12,13,1,5,17,0,12,1


In [76]:
from river import cluster
from river import stream
from river import tree

streamkmeans = cluster.STREAMKMeans(n_clusters=4, halflife=0.5, sigma=1.5, seed=0)

In [77]:
dt_scaler = StandardScaler();
scaled_features = dt_scaler.fit_transform(data_df.values)
scaled_features_df = pd.DataFrame(scaled_features, index=data_df.index, columns=data_df.columns)

In [78]:
for x in scaled_features_df.iterrows():
  data = {}
  for y in range(x[1].size):
    data[y] = x[1][y]
  streamkmeans = streamkmeans.learn_one(data)

In [79]:
streamkmeans.centers

{0: defaultdict(functools.partial(<bound method Random.gauss of <random.Random object at 0x555959311720>>, 0, 1.5),
             {0: -0.8444124213031913,
              1: 0.2625590295445621,
              2: 0.5568197121907725,
              3: -0.42215122778488534,
              4: -0.11719143737791782,
              5: 1.546710455705604,
              6: -0.5455328212007539,
              7: -0.13134654610865915,
              8: -1.0077307840790608,
              9: -0.35597799372425565,
              10: 0.5656786075604427}),
 1: defaultdict(functools.partial(<bound method Random.gauss of <random.Random object at 0x555959311720>>, 0, 1.5),
             {0: 0.31814095417168087,
              1: 0.8695427819269008,
              2: -0.3076568494614655,
              3: 1.4560510415042873,
              4: 1.1541603348605363,
              5: -0.2255537715108148,
              6: 1.5411626696873173,
              7: 1.0404580120415914,
              8: -0.9031962609054913,
           

In [80]:
classes = []
for x in scaled_features_df.iterrows():
  data = {}
  for y in range(x[1].size):
    data[y] = x[1][y]

  classes.append(streamkmeans.predict_one(data))
data_df['class'] = classes

In [81]:
data_df

,xpos,ypos,coin,emnemy,shot,enemy/shot,level,keyX,keyY,respawn_enemy,"""respawn_coin""",class
0,415.25,344.25,1,0,1,1,1,3,4,5,6,3
1,317.60,178.00,7,6,6,0,1,13,13,6,7,0
2,344.20,218.60,6,6,8,2,2,21,16,11,11,0
3,297.30,387.30,2,8,10,2,3,13,10,13,7,1
4,354.70,372.60,4,8,9,1,3,17,10,8,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...
382,472.50,536.00,0,9,11,2,4,8,0,9,0,1
383,385.00,536.00,2,7,8,1,5,15,0,12,7,1
384,227.50,536.00,0,7,8,1,5,14,0,7,0,1
385,435.40,536.00,1,12,13,1,5,17,0,12,1,1


In [82]:
model = tree.HoeffdingAdaptiveTreeClassifier(
  grace_period=100,
  split_confidence=1e-5,
  leaf_prediction='nb',
  nb_threshold=10,
  seed=0
)

metric = metrics.ROCAUC()

In [83]:
train_datas = []
for x in list(data_df.iterrows()):
  data = {}
  for y in range(x[1].size-1):
    data[y] = x[1][y]
  train_datas.append((data, x[1][-1]))
  # model.learn_one(data, x[1][-1])

In [84]:
evaluate.progressive_val_score(train_datas, model, metric)

ROCAUC: 0.976505

In [ ]:
joblib.dump(model, 'model.h5')

['model.h5']

In [ ]:
mean_collected_coin = list(data_df.groupby(['class']).mean()['coin'])
mean_enemies_kills = list(data_df.groupby(['class']).mean()['emnemy'])
print(mean_enemies_kills)
print(mean_collected_coin)

[3.849056603773585, 6.220779220779221, 1.9942528735632183, 13.433333333333334]
[0.9716981132075472, 3.857142857142857, 1.2873563218390804, 1.1]


In [85]:
sum_df = pd.DataFrame()
sum_df['xpos'] = list(data_df.groupby(['class']).mean()['xpos'])
sum_df['ypos'] = list(data_df.groupby(['class']).mean()['ypos'])
sum_df['coin'] = list(data_df.groupby(['class']).mean()['coin'])
sum_df['enemy'] = list(data_df.groupby(['class']).mean()['emnemy'])
sum_df['shot'] = list(data_df.groupby(['class']).mean()['shot'])
sum_df['shot-enemy'] = list(data_df.groupby(['class']).mean()['enemy/shot'])
sum_df['level'] = list(data_df.groupby(['class']).mean()['level'])
sum_df['keyX'] = list(data_df.groupby(['class']).mean()['keyX'])
sum_df['keyY'] = list(data_df.groupby(['class']).mean()['keyY'])
sum_df['respawn_enemy'] = list(data_df.groupby(['class']).mean()['respawn_enemy'])
sum_df['respawn_coin'] = list(data_df.groupby(['class']).mean()[' "respawn_coin"'])


In [86]:
sum_df

,xpos,ypos,coin,enemy,shot,shot-enemy,level,keyX,keyY,respawn_enemy,respawn_coin
0,286.918447,360.889078,2.456311,2.485437,6.009709,3.524272,1.970874,9.029126,6.640777,4.592233,4.592233
1,349.344565,466.273913,1.739130,10.108696,11.434783,1.326087,4.239130,14.250000,5.445652,12.826087,4.456522
2,324.020000,396.130000,1.600000,4.400000,5.100000,0.700000,4.200000,7.400000,3.400000,7.700000,5.100000
3,419.489560,428.959478,1.252747,2.236264,3.642857,1.406593,1.873626,8.236264,4.384615,4.395604,3.395604


In [87]:
labels = []
num_labels = len(mean_collected_coin)
for i in range(num_labels):
    if i % 2 == 0:
        sorted_index = np.argsort(mean_collected_coin)
    else:
        sorted_index = np.argsort(mean_enemies_kills)

    sorted_index = list(sorted_index)
    sorted_index.reverse()
    for max_index in sorted_index:
        if max_index not in labels:
            labels.append(max_index)
            break

print(labels)

LABELS = {
    labels[0]: 'Hardcore Achiever',
    labels[1]: 'Hardcore Killer',
    labels[2]: 'Casual Achiever',
    labels[3]: 'Casual Killer',
}
print(LABELS)

[1, 3, 2, 0]
{1: 'Hardcore Achiever', 3: 'Hardcore Killer', 2: 'Casual Achiever', 0: 'Casual Killer'}
